In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

In [4]:
#SENZA QUELLI CON MENO DI 40 PARTITE GIOCATE
# DATA PREPARATION
with open("player_stats.pkl", "rb") as f:
    total_stats = pickle.load(f)
#tolgo seasons che non ci servono
total_stats = total_stats[total_stats["SEASON_ID"] > 2003]
total_stats = total_stats[total_stats["SEASON_ID"] < 2022]
#sposto la colonna MVP per prima per comodita
cols = total_stats.columns.tolist()
cols = cols[-2:] + cols[:-2]
total_stats = total_stats[cols]

total_stats["GP"] = total_stats["GP"].astype('int')
total_stats["FGM"] = total_stats["FGM"].astype('int')
total_stats["FGA"] = total_stats["FGA"].astype('int')
total_stats["FG3M"] = total_stats["FG3M"].astype('int')
total_stats["FG3A"] = total_stats["FG3A"].astype('int')
total_stats["FTM"] = total_stats["FTM"].astype('int')
total_stats["FTA"] = total_stats["FTA"].astype('int')
total_stats["OREB"] = total_stats["OREB"].astype('int')
total_stats["DREB"] = total_stats["DREB"].astype('int')
total_stats["REB"] = total_stats["REB"].astype('int')
total_stats["AST"] = total_stats["AST"].astype('int')
total_stats["STL"] = total_stats["STL"].astype('int')
total_stats["BLK"] = total_stats["BLK"].astype('int')
total_stats["TOV"] = total_stats["TOV"].astype('int')
total_stats["PF"] = total_stats["PF"].astype('int')
total_stats["PTS"] = total_stats["PTS"].astype('int')

total_stats = total_stats[total_stats["GP"] > 40]


#divido il dataset
season_2021 = total_stats[total_stats["SEASON_ID"] == 2021]

#creo nuovo dataframe per i modelli
df = total_stats[total_stats["SEASON_ID"] < 2021]

#separo labels da features
labels = df['MVP']
features = df.drop('MVP', axis=1)
#divido train e test --> dobbiamo decidere grandezza test set


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.20, random_state=42)
#standardizzazione i nostri dati per anno


scaler = StandardScaler()

num_cols = df.columns
num_cols = num_cols.drop(["MVP", 'CONF_RANK', 'PLAYER_ID', 'SEASON_ID', 'TEAM_ID'])

train_data = pd.DataFrame()
train_data = pd.concat([train_data, train_features], axis=0, ignore_index=True)

test_data = pd.DataFrame()
test_data = pd.concat([test_data, test_features], axis=0, ignore_index=True)

for year in train_data["SEASON_ID"].unique().tolist():
    for col in num_cols.tolist():
        a = scaler.fit_transform(train_features.loc[train_features['SEASON_ID'] == year, col].to_numpy().reshape(-1, 1))
        train_data.loc[train_data['SEASON_ID'] == year, col] = scaler.fit_transform(train_features.loc[train_features['SEASON_ID'] == year, col].to_numpy().reshape(-1, 1))

        test_data.loc[test_data['SEASON_ID'] == year, col] = scaler.transform(test_features.loc[test_features['SEASON_ID'] == year, col].to_numpy().reshape(-1, 1))

train_l = pd.DataFrame()
train_l = pd.concat([train_l, train_labels], axis=0, ignore_index=True)

#correlation matrix solo del training e tolgo quelle piu correlate
train = pd.DataFrame()
train = pd.concat([train_l, train_data], axis=1, ignore_index=False)

train = train.rename(columns={0: "MVP"})

X_corr = train_data.corr()
corr_names = set()

for i in range(len(X_corr.columns)):
    for j in range(i):
        if abs(X_corr.iloc[j, i]) > 0.9:
            col = X_corr.columns[i]
            corr_names.add(col)

train_data.drop(columns=corr_names, inplace=True)
test_data.drop(columns=corr_names, inplace=True)

# le features tolte con correlazione maggiore di 0.9 sono:{'FG3A', 'FGA', 'FTA', 'PTS', 'REB'}

## SMOTE
train.drop(columns=corr_names, inplace=True)
training_data = train  #Mettere solo Train e non test

mvp = training_data["MVP"]

data_to_over = training_data.drop(["MVP", "PLAYER_ID", "SEASON_ID", "TEAM_ID", "PLAYER_AGE"], axis=1)

oversample = SMOTE(sampling_strategy=0.5, random_state=23)
X, y = oversample.fit_resample(data_to_over, mvp)

over_train_data = pd.concat([X, y], axis=1)


#Save Over_train_data
with open("Data/train_data_40.pkl", 'wb') as f:
    pickle.dump(over_train_data, f)

test_data["MVP"]  = list(test_labels)
with open("Data/test_data_40.pkl", 'wb') as f:
    pickle.dump(test_data, f)

# without all players (no <40 filter)

In [3]:
#SENZA QUELLI CON MENO DI 40 PARTITE GIOCATE
# DATA PREPARATION
with open("player_stats.pkl", "rb") as f:
    total_stats = pickle.load(f)
#tolgo seasons che non ci servono
total_stats = total_stats[total_stats["SEASON_ID"] > 2003]
total_stats = total_stats[total_stats["SEASON_ID"] < 2022]
#sposto la colonna MVP per prima per comodita
cols = total_stats.columns.tolist()
cols = cols[-2:] + cols[:-2]
total_stats = total_stats[cols]

total_stats["GP"] = total_stats["GP"].astype('int')
total_stats["FGM"] = total_stats["FGM"].astype('int')
total_stats["FGA"] = total_stats["FGA"].astype('int')
total_stats["FG3M"] = total_stats["FG3M"].astype('int')
total_stats["FG3A"] = total_stats["FG3A"].astype('int')
total_stats["FTM"] = total_stats["FTM"].astype('int')
total_stats["FTA"] = total_stats["FTA"].astype('int')
total_stats["OREB"] = total_stats["OREB"].astype('int')
total_stats["DREB"] = total_stats["DREB"].astype('int')
total_stats["REB"] = total_stats["REB"].astype('int')
total_stats["AST"] = total_stats["AST"].astype('int')
total_stats["STL"] = total_stats["STL"].astype('int')
total_stats["BLK"] = total_stats["BLK"].astype('int')
total_stats["TOV"] = total_stats["TOV"].astype('int')
total_stats["PF"] = total_stats["PF"].astype('int')
total_stats["PTS"] = total_stats["PTS"].astype('int')


#divido il dataset
season_2021 = total_stats[total_stats["SEASON_ID"] == 2021]

#creo nuovo dataframe per i modelli
df = total_stats[total_stats["SEASON_ID"] < 2021]

#separo labels da features
labels = df['MVP']
features = df.drop('MVP', axis=1)
#divido train e test --> dobbiamo decidere grandezza test set


train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.20, random_state=42)
#standardizzazione i nostri dati per anno


scaler = StandardScaler()

num_cols = df.columns
num_cols = num_cols.drop(["MVP", 'CONF_RANK', 'PLAYER_ID', 'SEASON_ID', 'TEAM_ID'])

train_data = pd.DataFrame()
train_data = pd.concat([train_data, train_features], axis=0, ignore_index=True)

test_data = pd.DataFrame()
test_data = pd.concat([test_data, test_features], axis=0, ignore_index=True)

for year in train_data["SEASON_ID"].unique().tolist():
    for col in num_cols.tolist():
        a = scaler.fit_transform(train_features.loc[train_features['SEASON_ID'] == year, col].to_numpy().reshape(-1, 1))
        train_data.loc[train_data['SEASON_ID'] == year, col] = scaler.fit_transform(train_features.loc[train_features['SEASON_ID'] == year, col].to_numpy().reshape(-1, 1))

        test_data.loc[test_data['SEASON_ID'] == year, col] = scaler.transform(test_features.loc[test_features['SEASON_ID'] == year, col].to_numpy().reshape(-1, 1))

train_l = pd.DataFrame()
train_l = pd.concat([train_l, train_labels], axis=0, ignore_index=True)

#correlation matrix solo del training e tolgo quelle piu correlate
train = pd.DataFrame()
train = pd.concat([train_l, train_data], axis=1, ignore_index=False)

train = train.rename(columns={0: "MVP"})

X_corr = train_data.corr()
corr_names = set()

for i in range(len(X_corr.columns)):
    for j in range(i):
        if abs(X_corr.iloc[j, i]) > 0.9:
            col = X_corr.columns[i]
            corr_names.add(col)

train_data.drop(columns=corr_names, inplace=True)
test_data.drop(columns=corr_names, inplace=True)

# le features tolte con correlazione maggiore di 0.9 sono:{'FG3A', 'FGA', 'FTA', 'PTS', 'REB'}

## SMOTE
train.drop(columns=corr_names, inplace=True)
training_data = train  #Mettere solo Train e non test

mvp = training_data["MVP"]

data_to_over = training_data.drop(["MVP", "PLAYER_ID", "SEASON_ID", "TEAM_ID", "PLAYER_AGE"], axis=1)

oversample = SMOTE(sampling_strategy=0.5, random_state=23)
X, y = oversample.fit_resample(data_to_over, mvp)

over_train_data = pd.concat([X, y], axis=1)


#Save Over_train_data
with open("Data/train_data.pkl", 'wb') as f:
    pickle.dump(over_train_data, f)

test_data["MVP"]  = list(test_labels)
with open("Data/test_data.pkl", 'wb') as f:
    pickle.dump(test_data, f)